In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz
from sklearn import preprocessing
from IPython.display import Image
import graphviz
import pydot
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import io
from google.colab import files
uploaded = files.upload()

Saving binary_classification_train.csv to binary_classification_train.csv


In [ ]:
df = pd.read_csv("/content/binary_classification_train.csv", sep="\t")
print(df)

       Label  Header                                           Sequence  \
0          1  O88483  MPAPTQLFFPLVRNCELSRIYGTACYCHHKHLCCSPPYIPQNRLRY...   
1          1  P25646  MRLGNAYAYCKPSQNVGLKLDLLRGLPGYVGHATSRINRLENQDNY...   
2          1  O53896  MAKLARVVGLVQEEQPSDMTNHPRYSPPPQQPGTPGYAQGQQQTYS...   
3          1  Q44238  MNKLAVLYAEHIATLQKRTREIIERENLDGVVFHSGQAKRQFLDDM...   
4          1  Q9QYJ6  MEDGPSNNASCFRRLTECFLSPSLTDEKVKAYLSLHPQVLDEFVSE...   
...      ...     ...                                                ...   
76313      0  P06568  MKRKTAVKWLAVLAGAGLLYWGNKTYLNVSPKEIRVWVLSFGVFAP...   
76314      0  P55731  MHGGIQRISLRGSMLMPPQSNWQDNRFSSVAAQVDVPRAHIRGHARFSI   
76315      0  Q62901  MTADKDKDKDKEKDRDRDRDRERDKRDKARESENARPRRSCTLEGG...   
76316      0  P05790  MRVKTFVILCCALQYVAYTNANINDFDEDYFGSDVTVQSSNTTDEI...   
76317      0  Q9P258  MPRKKAAAAAWEEPSSGNGTARAGPRKRGGPAGRKRERPERCSSSS...   

       Sequence length        Mass Most frequent aa  Last aa is H  \
0                  538   61150

# New Section

In [ ]:
#Convert non numeric data into numeric data
non_numeric_set=[df['Most frequent aa']]
le = preprocessing.LabelEncoder()
after_processing=[]
for s in non_numeric_set:
  after_processing.append(le.fit_transform(s))
df['Most frequent aa']=after_processing[0]

print(df)



       Label  Header                                           Sequence  \
0          1  O88483  MPAPTQLFFPLVRNCELSRIYGTACYCHHKHLCCSPPYIPQNRLRY...   
1          1  P25646  MRLGNAYAYCKPSQNVGLKLDLLRGLPGYVGHATSRINRLENQDNY...   
2          1  O53896  MAKLARVVGLVQEEQPSDMTNHPRYSPPPQQPGTPGYAQGQQQTYS...   
3          1  Q44238  MNKLAVLYAEHIATLQKRTREIIERENLDGVVFHSGQAKRQFLDDM...   
4          1  Q9QYJ6  MEDGPSNNASCFRRLTECFLSPSLTDEKVKAYLSLHPQVLDEFVSE...   
...      ...     ...                                                ...   
76313      0  P06568  MKRKTAVKWLAVLAGAGLLYWGNKTYLNVSPKEIRVWVLSFGVFAP...   
76314      0  P55731  MHGGIQRISLRGSMLMPPQSNWQDNRFSSVAAQVDVPRAHIRGHARFSI   
76315      0  Q62901  MTADKDKDKDKEKDRDRDRDRERDKRDKARESENARPRRSCTLEGG...   
76316      0  P05790  MRVKTFVILCCALQYVAYTNANINDFDEDYFGSDVTVQSSNTTDEI...   
76317      0  Q9P258  MPRKKAAAAAWEEPSSGNGTARAGPRKRGGPAGRKRERPERCSSSS...   

       Sequence length        Mass  Most frequent aa  Last aa is H  \
0                  538   6115

In [ ]:
#Define the features and output:
y = np.array(df['Label'])
X = df.drop(['Label','Header','Sequence'], axis=1)
#Split data into train an test, with test size of 20%:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)
#Build the model:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
#Evaluate the model:
print("accuracy on training set: %f" % rf.score(X_train, y_train))
print("accuracy on test set: %f" % rf.score(X_test, y_test))

accuracy on training set: 0.999951
accuracy on test set: 0.898716


In [ ]:
#Using only 5 nodes:
rf_5 = RandomForestClassifier(max_depth=5)
rf_5.fit(X_train, y_train)
# Pull out one tree from the forest:
tree = rf_5.estimators_[5]
# Export the image to a dot file:
export_graphviz(tree, out_file = 'tree.dot', feature_names = X.columns, rounded = True, precision = 1)
# Use dot file to create a graph:
(graph, ) = pydot.graph_from_dot_file('tree.dot')
# Write graph to a png file:
graph.write_png('tree_rf.png')